In [ ]:
import glob
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import imutils
from rasterio import features

In [ ]:
# Salvar imagens com as dimensões 640 x 640 
def save_resize_img(dir_path):
    for image_path in glob.glob(dir_path + "/*"):
        img = cv2.imread(image_path)
        img = cv2.resize(img, (640, 640))
        cv2.imwrite(image_path, img)
    return 



# Create yolo labels

In [ ]:
# label encoding
def label_encoding(x):
    labels = {'Disc':0, 'Cup':1}
    return labels[x]    
    
def extract_yolo(path, img_name, path_masks):
    
    if not os.path.exists(path):
        os.makedirs(path)
    
    j = 'Disc'
    
    for i in range(len(img_name)):
#         mask_image = cv2.imread(path_masks + img_name[i] + '_ODsegSoftmap.png') # drishti-gs
        mask_image = cv2.imread(path_masks + 'disc_' + img_name[i] + '.jpg') #g1020
#         mask_image = cv2.imread(path_masks + img_name[i] + '_disc_exp1.jpg') # papila

        
        mask_image = cv2.cvtColor(mask_image, cv2.COLOR_BGR2GRAY)
#         mask_image = cv2.resize(mask_image, (640, 640))
        
        # Convert to binary image using thresholding
        mask_image = cv2.threshold(mask_image, 50, 255, cv2.THRESH_BINARY)[1]  # cup optic 
        
        cords = list(features.shapes(mask_image, mask=(mask_image >0)))[0][0]['coordinates'][0]
        label_line = str(label_encoding(j))+ ' ' + ' '.join([f'{int(cord[0])/mask_image.shape[1]} {int(cord[1])/mask_image.shape[0]}' for cord in cords])

        # Plotar contornos das coordenadas nas máscaras
#         plt.imshow(mask_image, cmap='gray')
#         x_coords = [coord[0] for coord in cords]
#         y_coords = [coord[1] for coord in cords]
#         plt.plot(x_coords, y_coords, 'ro', markersize=2)  
#         plt.show()

        
        # Save bounding boxes to a file in YOLOv8 format
        with open(path + '/' + img_name[i] + '.txt', 'a') as f:
            f.write(label_line)

        # Remove duplicate
        with open(path + '/' + img_name[i] + '.txt', 'r') as input_file:
            lines = input_file.readlines()
            # Remove duplicates from the list of lines
            unique_lines = sorted(list(set(lines)))
            # Open the output file and write the unique lines to it
            with open(path + '/' + img_name[i] + '.txt', 'w') as output_file:
                output_file.writelines(unique_lines)

    return label_line



In [ ]:
# Ler o nome de todas as imagens de treino e teste
def image_name(dir_path):
    img_name = []
    for image_path in glob.glob(dir_path + "/*"):
        file = image_path.split("\\")[-1][:-4]
        if image_path.split("\\")[-1][-4:] == '.jpg':
            img_name.append(file)
    return img_name



img_name_train = image_name('D:/Glaucoma/Banco_de_dados/G1020/images/')
# img_name_val = image_name('C:/Users/debora.assis/Documents/Multi_Classe/papila/Images/0_Normal/')
# img_name_val2 = image_name('C:/Users/debora.assis/Documents/Multi_Classe/papila/Images/1_Glaucoma/')
# img_name_test = image_name('C:/Users/debora.assis/Documents/Multi_Classe/papila/Images/2_Suspeito/')



In [ ]:
# pastas que estão localizadas as máscaras 
path_masks_train = 'D:/Glaucoma/Banco_de_dados/G1020/masks/'
# path_masks_val = 'C:/Users/debora.assis/Documents/Multi_Classe/papila/Masks_sep/0_Normal/'
# path_masks_val2 = 'C:/Users/debora.assis/Documents/Multi_Classe/papila/Masks_sep/1_Glaucoma/'
# path_masks_test = 'C:/Users/debora.assis/Documents/Multi_Classe/papila/Masks_sep/2_Suspeito/'


# pastas criadas que estarão as saídas das labels
path_output_train = 'C:/Users/debora.assis/Documents/REFUGE/train/labels/'
# path_output_val = 'C:/Users/debora.assis/Documents/REFUGE/train/labels/'
# path_output_val2 = 'C:/Users/debora.assis/Documents/REFUGE/train/labels/'
# path_output_test = 'C:/Users/debora.assis/Documents/REFUGE/train/labels/'


yolo_train = extract_yolo(path_output_train, img_name_train, path_masks_train)
# yolo_val = extract_yolo(path_output_val, img_name_val, path_masks_val)
# yolo_val2 = extract_yolo(path_output_val2, img_name_val2, path_masks_val2)
# yolo_test = extract_yolo(path_output_test, img_name_test, path_masks_test)

In [ ]:
# Copiar arquivos de uma pasta para outra

In [ ]:
import os
from PIL import Image
import shutil

path = 'C:/Users/debora.assis/Documents/YOLOv5_bounding_box_from_masks/PrepareData/teste_externo/'
pasta_destino = 'C:/Users/debora.assis/Documents/REFUGE/test_drisht/images'

arquivos_na_pasta = os.listdir(path)
nomes_de_imagens = [arquivo for arquivo in arquivos_na_pasta if arquivo.lower().endswith(('.jpg', '.jpeg', '.png', '.gif', '.bmp'))]

# Itera pelos arquivos na pasta de origem
for arquivo in arquivos_na_pasta:
    caminho_completo = os.path.join(path, arquivo)
    
    # Verifica se o arquivo é uma imagem
    if arquivo.lower().endswith(('.jpg', '.jpeg', '.png', '.gif', '.bmp')):
        # Abre a imagem com o Pillow
        imagem = Image.open(caminho_completo)
        
        # Redimensiona para 640x640 pixels
        imagem_redimensionada = imagem.resize((640, 640))
        
        # Define o caminho de destino com o mesmo nome do arquivo original
        caminho_destino = os.path.join(pasta_destino, arquivo)
        
        # Salva a imagem redimensionada na pasta de destino
        imagem_redimensionada.save(caminho_destino)
        print(f'{arquivo} redimensionado e copiado para {caminho_destino}')


In [ ]:
# Mover arquivos
import shutil

path = 'C:/Users/debora.assis/Documents/data/train/labels/'

arquivos_na_pasta = os.listdir(path)
nomes_de_imagens = [arquivo for arquivo in arquivos_na_pasta if arquivo.lower().endswith(('.jpg', '.jpeg', '.png', '.gif', '.bmp', '.txt'))]

for i in nomes_de_imagens:
    img_path =  os.path.join('C:/Users/debora.assis/Documents/REFUGE/train/labels/', i)
    print(img_path)
    shutil.move(img_path, 'C:/Users/debora.assis/Documents/apagar/train/labels/'+ i)


In [ ]:
# path_masks = 'C:/Users/debora.assis/Documents/Multi_Classe/papila/Masks_sep/0_Normal/'
# mask_image = cv2.imread(path_masks +'RET027OD_disc_exp1.jpg') # papila

path_masks = 'D:/Glaucoma/Banco_de_dados/G1020/masks/'
mask_image = cv2.imread(path_masks + 'disc_image_105' + '.jpg') #g1020

mask_image = cv2.cvtColor(mask_image, cv2.COLOR_BGR2GRAY)
#         mask_image = cv2.resize(mask_image, (640, 640))

mask_image = cv2.threshold(mask_image, 50, 255, cv2.THRESH_BINARY)[1]  # cup optic 

cords = list(features.shapes(mask_image, mask=(mask_image >0)))[0][0]['coordinates'][0]
label_line = str(0)+ ' ' + ' '.join([f'{int(cord[0])/mask_image.shape[1]} {int(cord[1])/mask_image.shape[0]}' for cord in cords])

# Plotar contornos das coordenadas nas máscaras
plt.imshow(mask_image, cmap='gray')
x_coords = [coord[0] for coord in cords]
y_coords = [coord[1] for coord in cords]
plt.plot(x_coords, y_coords, 'ro', markersize=2)  
plt.show()


In [ ]:
cords

In [ ]:
label_line

In [ ]:
cords